In [77]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, SimpleRNN,Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import GRU
import tensorflow as tf


In [78]:
# Load the dataset
df = pd.read_csv('..//..//dataset//final_dataset.csv')  

# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#skip normalization
X_scaled=X

# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [79]:
print(X_train.shape, y_train.shape,X_scaled.shape)

(320000, 1, 6) (320000,) (400000, 1, 6)


# Model

In [80]:
# Split dataset manually (80% Train, 20% Validation)
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Convert to tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_split, y_train_split))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))

# Batch and prefetch for efficiency
BATCH_SIZE = 128
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


def build_drnn_lstm():
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

    # Add multiple SimpleRNN layers for Deep RNN
    model.add(SimpleRNN(units=50, return_sequences=True, activation='relu'))
    model.add(Dropout(0.1))
    



    model.add(SimpleRNN(units=50, activation='relu',return_sequences=True))
    model.add(Dropout(0.1))

    model.add(LSTM(units=50, activation='relu',return_sequences=True))
    model.add(Dropout(0.1))

    model.add(LSTM(units=50))
    model.add(Dropout(0.1))



    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])

    return model



In [81]:
import time 
model = build_drnn_lstm()

# Train the model
start_time = time.time()
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,  # Use manually split validation set
    verbose=1
)
end_time = time.time()



Epoch 1/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 38s 12ms/step - loss: 0.2042 - mae: 0.1417 - mse: 0.0674 - val_loss: 0.0010 - val_mae: 4.7708e-04 - val_mse: 2.9653e-04
Epoch 2/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 23s 11ms/step - loss: 0.0043 - mae: 0.0022 - mse: 0.0010 - val_loss: 2.8624e-04 - val_mae: 2.2880e-04 - val_mse: 6.6399e-05
Epoch 3/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 23s 11ms/step - loss: 0.0022 - mae: 0.0011 - mse: 5.2049e-04 - val_loss: 1.4320e-04 - val_mae: 1.1969e-04 - val_mse: 2.9401e-05
Epoch 4/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - loss: 0.0013 - mae: 6.9863e-04 - mse: 3.3648e-04 - val_loss: 8.2876e-05 - val_mae: 7.6417e-05 - val_mse: 1.0499e-05
Epoch 5/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 9.5351e-04 - mae: 4.7571e-04 - mse: 2.2193e-04 - val_loss: 4.2609e-05 - val_mae: 3.7798e-05 - val_mse: 7.5927e-06
Epoch 6/50
1998/2000 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9.1655e-04 - mae: 4.2196e-04 - mse: 2.1031e-04

KeyboardInterrupt: 

In [ ]:
# Calculate total training time
total_time = end_time - start_time  # in seconds

# Number of training samples
num_samples = len(X_train)
aes=total_time/num_samples
# Print results
print(f"Total Training Time: {total_time:.2f} seconds")
aes_ms = aes/1000
print(f"AES: {aes_ms:.2f} ms/sample")

Total Training Time: 63.81 seconds
AES: 0.00 ms/sample


In [ ]:
import time

# Number of samples in the test dataset
num_samples = X_test.shape[0]

# Measure inference time
start_time = time.time()
y_pred = model.predict(X_test)  # Model makes predictions
end_time = time.time()

# Compute AES
total_prediction_time = end_time - start_time
aes = (total_prediction_time / num_samples) * 1e6  # Time per sample
print(f"Total Prediction Time: {total_prediction_time:.6f} seconds")
print(f"AES (Inference Time per Sample): {aes:.6f} µs/sample")
aes_ms = aes/1000
print(f"AES (Inference Time per Sample): {aes_ms:.6f} ms/sample")

2500/2500 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step
Total Prediction Time: 11.161090 seconds
AES (Inference Time per Sample): 139.513621 µs/sample
AES (Inference Time per Sample): 0.139514 ms/sample


In [ ]:

from sklearn.metrics import mean_squared_error
import numpy as np

# Make predictions
y_pred = model.predict(X_test)
print(y_pred.shape)
# Calculate MSE


2500/2500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step
(80000, 1)


In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')

Mean Squared Error (MSE): 0.0000127583534777
Root Mean Squared Error (RMSE): 0.0035718837435859


In [ ]:
# #visualizing the loss and accuracy
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10,5))
# plt.subplot(1,2,1)
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train','validation'],loc='upper left')

# plt.subplot(1,2,2)
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train','validation'],loc='upper left')
# plt.show()


In [ ]:
# model.save('..//..//models//base_paper_model.keras')  